In [1]:
import pandas as pd
import numpy as py
from preprocess_helper import PorterStemmer
import os
import re
from bs4 import BeautifulSoup
from string import punctuation
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/wangsimin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/wangsimin/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
class Dataset_Preprocess:
    def __init__(self):
        self.stopwords_list = None
        self.tag_list = None
        self.reservedkeywords = None
    
    def create_reservedkeywords(self, path, num_words = 100):
        df = pd.read_csv(path)
        df.sort_values(by=['Count'], ascending=False, inplace=True, ignore_index=True)
        self.tag_list = list(df["Tag"].values)
        reservedkeywords_extra = ["c#","f#","c++","node.js","nodejs",".json",".js",".net","objective-c",
                                  "asp.net","ruby-on-rails","angular.js"]
        self.reservedkeywords = list(df["Tag"].values)[:num_words]
        self.reservedkeywords.extend(reservedkeywords_extra)
        self.reservedkeywords = set(self.reservedkeywords)
        #print(self.reservedkeywords)
    
    def create_stopwords(self):
        self.stopwords_list = stopwords.words('english')
        stop_words_extra = ["i'd","sometime","sometimes","something","someone","somebody","anything","anyone","anybody",
                            "everytime","everything","everyone","everybody","e.g.","e.g","e.g.,","i.e.","i.e","i.e.,","love",
                            "know","'s","wonder"]
        self.stopwords_list.extend(stop_words_extra)
        stopwords_unsure_list = set(self.stopwords_list).intersection(set(self.tag_list))
        self.stopwords_list = set(self.stopwords_list).difference(stopwords_unsure_list)
        #print(len(self.stopwords_list),"stopwords",self.stopwords_list)
    
    def remove_non_ascii(self,sentence):
        return ''.join(char for char in sentence if ord(char) < 128)
    
    def html_Filter(self, sentence):
        sentence = BeautifulSoup(sentence, "lxml").text
        #print("after html_Filter",sentence)
    
        return sentence
    
    def keywords_transform(self, sentence):
        sentence = sentence.lower()
        sentence = sentence.replace("node js","node.js")
        sentence = sentence.replace("objective c","objective-c")
        sentence = sentence.replace("ruby on rails","ruby-on-rails")
        sentence = sentence.replace("angular js","angular-js")
        
        return sentence
    
    def POSTag_generation(self, sentence):
        sentence_clean = []
        
        #split text into indiviual sentences for better pos tagging
        sentence_list = sent_tokenize(sentence)

        for sentence in sentence_list:
            text = word_tokenize(sentence)
            text_new = []
            skip_num = False
            
            for i,word in enumerate(text):
                #keep "c#" and "F#"
                
                if skip_num:
                    skip_num = False
                    continue
                
                if word == "c" or word == "f":
                    if i+1 <len(text) and text[i+1] == "#":
                        text_new.append(word+"#")
                        skip_num = True
                    else:
                        text_new.append(word)
                else:
                    text_new.append(word)
            
            print(text_new)
            pos_words = nltk.pos_tag(text_new)
         
            for i,pos_word in enumerate(pos_words):
                #keep "c#" and "F#"
                word = pos_word[0]
                if word == "c" or word == "f":
                    if i+1 <len(pos_words) and pos_words[i+1][0] == "#":
                        sentence_clean.append(word+"#")
                                 
                sentence_clean.append(word)
         
        #return re.sub(r'\W+', ' ', ' '.join(sentence_clean))
        
        #print("after POSTag_Removal",' '.join(sentence_clean))
        return ' '.join(sentence_clean)
    
    def remove_specialchar(self, sentence, char_to_keep = {'#','+','.','-'}):
        punct_set = set(punctuation).difference(char_to_keep)

        for i in punct_set:
            # Replace the special character with an empty string
            sentence=sentence.replace(i," ")
        
        return sentence
        
    def sentence_stem(self, sentence):
        p = PorterStemmer()
        output = ""

        for token in sentence.split(' '):
            if token.isalnum():
                output += p.stem(token, 0,len(token)-1)+' '
            elif token in self.reservedkeywords:
                output += token+' '
        
        #print("after sentence_stem", output.strip())
        return output.strip()
    
    def load_dataset(self, path, data_selectedId=None):
        # creating cleaned input, output pairs
        print("Start loading data...")
        allfiles = os.listdir(path)
        
        if data_selectedId == None:
            data_selectedId = [i for i in range(len(allfiles))]
        
        questions = []
        
        for filename in data_selectedId:
            text = ""
            #retreive qa_text
            try:
                with open(path+str(filename+1), 'r', encoding="utf-8") as f:
                    for line in f.readlines():
                        line = line.replace("\n","")
                        line = line.strip()
                        if line == "":
                            continue

                        text += line+" "
    
                questions.append(text.strip())
            except:
                print(str(filename+1),"is missing!")
            
            print("Loaded",len(questions),)
        return questions

    def call(self, dataset = None, keywords = None):
        data_clean = []
        tag_path = "../StackExchange/final_data/tag_dict.csv"
        
        if dataset == None:
            file_path = "../StackExchange/final_data/rawdata/"
            with open("../StackExchange/final_data/selected_id.txt", 'r') as f:
                all_ids = f.readlines()

            data_selectedId = sorted([int(tid.replace("\n","")) for tid in all_ids])
            
            dataset = load_dataset(file_path, data_selectedId)
        
        #initialize tag_list
        if keywords == None:
            self.create_reservedkeywords(tag_path)
        else:
            df = pd.read_csv(tag_path)
            self.tag_list = list(df["Tag"].values)
            self.reservedkeywords = keywords
        
        #initialize stopwords
        self.create_stopwords()
        
        for sentence in dataset:
            sentence = self.html_Filter(sentence)
            sentence = self.remove_non_ascii(sentence)
            sentence = self.keywords_transform(sentence)
            sentence = self.POSTag_generation(sentence)
            #sentence = self.remove_specialchar(sentence)
            #sentence = self.sentence_stem(sentence)
            
            data_clean.append(sentence.strip())
        
        return data_clean